# Default GAN

## Import libraries

In [ ]:
import math
import numpy as np
import librosa
from keras.layers import Input, Dense
from keras.models import Model
import soundfile as sf
import os

## init

In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # 1 = INFO, 2 = WARNING, 3 = ERROR
from keras.models import load_model

def calculate_psnr(original_signal, denoised_signal):  # Calculate PSNR
    original_signal = original_signal.astype(float)
    denoised_signal = denoised_signal.astype(float)
    mse = np.mean((original_signal - denoised_signal) ** 2)
    if mse == 0:
        return float('inf')
    max_pixel = np.max(original_signal)
    psnr = 20 * math.log10(max_pixel / math.sqrt(mse))
    return psnr, mse

# Load audio data
def load_audio(file_path):
    audio, sr = librosa.load(file_path, sr=16000)
    return audio, sr

## Generator Model

In [ ]:
def build_generator():
    input_layer = Input(shape=(1,))
    x = Dense(128, activation='relu')(input_layer)
    x = Dense(64, activation='relu')(x)
    output_layer = Dense(1, activation='tanh')(x)
    generator = Model(input_layer, output_layer)
    return generator

## Discriminator model

In [ ]:
def build_discriminator():
    input_layer = Input(shape=(1,))
    x = Dense(64, activation='relu')(input_layer)
    x = Dense(128, activation='relu')(x)
    output_layer = Dense(1, activation='sigmoid')(x)
    discriminator = Model(input_layer, output_layer)
    discriminator.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
    return discriminator

## Full Model

In [ ]:
# Build models
generator = build_generator()
discriminator = build_discriminator()

# GAN model
discriminator.trainable = False
gan_input = Input(shape=(1,))
generated_audio = generator(gan_input)
gan_output = discriminator(generated_audio)
gan = Model(gan_input, gan_output)
gan.compile(optimizer='adam', loss='mse')

## Learn

In [ ]:

# Training parameters
epochs = 1000
batch_size = 128

# Batch training with all files
clean_files = [f for f in os.listdir('./clean') if f.endswith('.wav')]
noisy_files = [f for f in os.listdir('./15dB') if f.endswith('.wav')]

for epoch in range(epochs):
    for clean_file, noisy_file in zip(clean_files, noisy_files):
        clean_audio, sr = load_audio(os.path.join('./clean', clean_file))
        noisy_audio, sr = load_audio(os.path.join('./15dB', noisy_file))

        # Ensure same length
        min_length = min(len(clean_audio), len(noisy_audio))
        clean_audio = clean_audio[:min_length]
        noisy_audio = noisy_audio[:min_length]

        # Normalize data
        clean_audio = (clean_audio - np.mean(clean_audio)) / np.std(clean_audio)
        noisy_audio = (noisy_audio - np.mean(noisy_audio)) / np.std(noisy_audio)

        # Training data
        X_train = noisy_audio.reshape(-1, 1)
        y_train = clean_audio.reshape(-1, 1)

        # Train discriminator
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        noisy_batch = X_train[idx]
        clean_batch = y_train[idx]